In [13]:
import tweepy
from tweepy import OAuthHandler
import pickle as p
import re
import json
from collections import defaultdict
import random
from numpy import arange, where, isin

In [9]:
twitter_keys=p.load(open('tk.pkl','rb'))

In [10]:

 
auth = OAuthHandler(twitter_keys['consumer_key'], twitter_keys['consumer_secret'])
auth.set_access_token(twitter_keys['access_token'], twitter_keys['access_secret'])
 
api = tweepy.API(auth)

In [11]:
def limit_handled(cursor):
    while True:
        try:
            yield cursor.next()
        except tweepy.RateLimitError:
            time.sleep(15 * 60)
            

In [14]:
hold=[]
with open('test.txt','w') as f:
    json.dump([], f)
for i,status in enumerate(limit_handled(tweepy.Cursor(api.user_timeline,'realDonaldTrump').items(1000))):
    hold.append(status._json)
    if i%100==0:
        with open('test.txt','r') as f:
            a=json.load(f)+hold
        with open('test.txt','w') as f:
            json.dump(a, f)
        hold=[]

In [15]:
trump_tweets=json.load(open('test.txt','r'))

In [ ]:
def retweet_filter(tweet):
    

In [16]:
def drop_links(tweet_words):
    try:
        cutoff=where([x=='https' for x in tweet_words])[0][0]
        return tweet_words[:cutoff]
    except IndexError:
        return tweet_words

In [17]:
def tweets_to_list(tweets):
    tweet_words=[]
    for tweet in tweets:
        if tweet['text'][:2]!='RT':
            tweet_words.extend(drop_links(re.findall(r"[\w']+|[.,!?;]",tweet['text'])))
    return tweet_words

In [18]:
trump_words=tweets_to_list(trump_tweets)


In [204]:
def generate_using_ngrams(n,words):
    '''
    Generate a sentence using the ngrams method. 
    
    Params:
    n: Words is broken into seqences of length n, n-1 words are then used to predict the next word.
    words: A list of words that generated sentences will be based on.
    '''
    
    if not n>1:raise ValueError("n must be at least 2")
        
    ngrams=zip(*[words[i:] for i in arange(n)])
    transitions=defaultdict(list)
    starts=[]
    
    for ngram in ngrams:
        if ngram[0] in ['.','!','?']:
            starts.append(ngram[1:-1])
        transitions[ngram[:-1]].append(ngram[-1])
    
    
    current= random.choice(starts)
    prev='.'
    result=[*current]

    while True:
        next_word_candidates=transitions[(prev,*current)]
        next_word = random.choice(next_word_candidates)
        prev, current = current[0], current[1:]+(next_word,)
        result.append(current[-1])
        if current[-1] in ['.','!','?']: return ' '.join(result)

In [212]:
generate_using_ngrams(3,trump_words)

'If we can do much better as a National Enquirer article .'

In [327]:
def generate_using_nxmgrams(n,m,words):
    '''
    Generate a sentence using the ngrams method. 
    
    Params:
    n: Words is broken into seqences of length n, n-1 words are then used to predict the next word.
    words: A list of words that generated sentences will be based on.
    '''
    
    if not n>1:raise ValueError("n must be at least 2")
        
    nxmgrams=zip(*[words[i:] for i in arange(n+m)])
    transitions=defaultdict(list)
    starts=[]
    
    for nxmgram in nxmgrams:
        if nxmgram[0] in ['.','!','?']:
            starts.append(nxmgram[1:-m])
        transitions[nxmgram[:-m]].append(nxmgram[-m:])
    
    current= random.choice(starts)
    prev='.'
    result=[*current]
    while True:
        print(prev,current)
        next_word_candidates=transitions[(prev,*current)]
        next_word = random.choice(next_word_candidates)
        prev = current[m-n]
        if n-1-m==0:
            current = (*next_word,)
        else:
            current =  current[m+1-n:]+(*next_word,)
        for c in current[-m:]: result.append(c)
        print(result)
        if current[-1] in ['.','!','?']: return ' '.join(result)

In [342]:
generate_using_nxmgrams(3,2,trump_words)

. ('We', 'pledge')
['We', 'pledge', 'our', 'solidarity']
pledge ('our', 'solidarity')
['We', 'pledge', 'our', 'solidarity', 'with', 'France']
solidarity ('with', 'France')
['We', 'pledge', 'our', 'solidarity', 'with', 'France', 'against', 'terror']
France ('against', 'terror')
['We', 'pledge', 'our', 'solidarity', 'with', 'France', 'against', 'terror', '.', 'It']
terror ('.', 'It')
['We', 'pledge', 'our', 'solidarity', 'with', 'France', 'against', 'terror', '.', 'It', 'was', 'a']
It ('was', 'a')
['We', 'pledge', 'our', 'solidarity', 'with', 'France', 'against', 'terror', '.', 'It', 'was', 'a', 'great', 'honor']
a ('great', 'honor')
['We', 'pledge', 'our', 'solidarity', 'with', 'France', 'against', 'terror', '.', 'It', 'was', 'a', 'great', 'honor', 'to', 'welcome']
honor ('to', 'welcome')
['We', 'pledge', 'our', 'solidarity', 'with', 'France', 'against', 'terror', '.', 'It', 'was', 'a', 'great', 'honor', 'to', 'welcome', 'Prime', 'Minister']
welcome ('Prime', 'Minister')
['We', 'pledge'

'We pledge our solidarity with France against terror . It was a great honor to welcome Prime Minister Najib Abdul Razak of Malaysia and his distinguished delegation to th Congratulations to Eric amp ; Lara on the birth of their son , Eric Luke Trump this morning !'